In [ ]:
import json
import os
from typing import List

class Fruit:
    def __init__(self, name: str, price: float, quantity: int):
        self.name = name
        self.price = price
        self.quantity = quantity
    
    def to_dict(self):
        return {
            "name": self.name,
            "price": self.price,
            "quantity": self.quantity
        }
    
    @classmethod
    def from_dict(cls, data: dict):
        return cls(data["name"], data["price"], data["quantity"])

class FruitManager:
    def __init__(self, file_path: str = "fruits.json"):
        self.file_path = file_path
        self.fruits = self._load_data()
    
    def _load_data(self) -> List[Fruit]:
        """从JSON文件加载数据"""
        if not os.path.exists(self.file_path):
            return []
        
        try:
            with open(self.file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                return [Fruit.from_dict(item) for item in data]
        except (json.JSONDecodeError, FileNotFoundError):
            return []
    
    def _save_data(self):
        """保存数据到JSON文件"""
        with open(self.file_path, "w", encoding="utf-8") as f:
            json.dump([fruit.to_dict() for fruit in self.fruits], f, indent=4, ensure_ascii=False)
    
    def add_fruit(self, fruit: Fruit) -> bool:
        """添加水果，如果已存在则增加数量"""
        for existing in self.fruits:
            if existing.name.lower() == fruit.name.lower():
                existing.quantity += fruit.quantity
                self._save_data()
                print(f"水果 {fruit.name} 已存在，已增加 {fruit.quantity} 个库存")
                return True
        
        self.fruits.append(fruit)
        self._save_data()
        return True
    
    def remove_fruit(self, fruit_name: str) -> bool:
        """删除水果"""
        initial_length = len(self.fruits)
        self.fruits = [f for f in self.fruits if f.name.lower() != fruit_name.lower()]
        if len(self.fruits) < initial_length:
            self._save_data()
            return True
        return False
    
    def update_fruit(self, fruit_name: str, new_price: float, new_quantity: int) -> bool:
        """更新水果信息"""
        for fruit in self.fruits:
            if fruit.name.lower() == fruit_name.lower():
                fruit.price = new_price
                fruit.quantity = new_quantity
                self._save_data()
                return True
        return False
    
    def get_all_fruits(self) -> List[Fruit]:
        """获取所有水果"""
        return self.fruits
    
    def get_fruit(self, fruit_name: str) -> Fruit:
        """获取指定水果"""
        for fruit in self.fruits:
            if fruit.name.lower() == fruit_name.lower():
                return fruit
        return None
    
    def sort_by_price(self) -> List[Fruit]:
        """按价格排序"""
        return sorted(self.fruits, key=lambda x: x.price)



# 主程序
def display_menu():
    print("\n" + "*" * 35 + "水果超市" + "*" * 35)
    print("1. 查询全部水果")
    print("2. 添加水果")
    print("3. 删除水果")
    print("4. 更新水果信息")
    print("5. 按价格排序")
    print("6. 查询指定水果")
    print("7. 退出系统")
    print("*" * 77)

def main():
    manager = FruitManager()
    
    while True:
        display_menu()
        choice = input("请输入您的选择：")
        
        if choice == "1":
            print("\n========== 查询全部水果信息 ==========")
            fruits = manager.get_all_fruits()
            if not fruits:
                print("水果超市里还没有水果")
            else:
                for fruit in fruits:
                    print(f"{fruit.name}：{fruit.price}元，{fruit.quantity}个")
        
        elif choice == "2":
            print("\n========== 上架水果 ==========")
            name = input("请输入水果名称：").strip()
            try:
                price = float(input("请输入水果价格："))
                quantity = int(input("请输入水果数量："))
                if price <= 0 or quantity <= 0:
                    print("价格和数量必须大于0")
                    continue
                
                fruit = Fruit(name, price, quantity)
                if manager.add_fruit(fruit):
                    print(f"{name} 水果添加成功")
            except ValueError:
                print("输入无效，请确保价格是数字，数量是整数")
        
        elif choice == "3":
            print("\n========== 下架水果 ==========")
            name = input("请输入要删除的水果名称：").strip()
            if manager.remove_fruit(name):
                print(f"{name} 水果删除成功")
            else:
                print(f"{name} 水果不存在")
        
        elif choice == "4":
            print("\n========== 更新水果信息 ==========")
            name = input("请输入要更新的水果名称：").strip()
            try:
                new_price = float(input("请输入新的水果价格："))
                new_quantity = int(input("请输入新的水果数量："))
                if new_price <= 0 or new_quantity <= 0:
                    print("价格和数量必须大于0")
                    continue
                
                if manager.update_fruit(name, new_price, new_quantity):
                    print(f"{name} 水果信息更新成功")
                else:
                    print(f"{name} 水果不存在")
            except ValueError:
                print("输入无效，请确保价格是数字，数量是整数")
        
        elif choice == "5":
            print("\n========== 按价格升序排列 ==========")
            fruits = manager.sort_by_price()
            if not fruits:
                print("水果超市里还没有水果")
            else:
                print("按照价格排序后的水果信息：")
                for fruit in fruits:
                    print(f"{fruit.name}：{fruit.price}元，{fruit.quantity}个")
        
        elif choice == "6":
            print("\n========== 查询指定水果 ==========")
            name = input("请输入要查询的水果名称：").strip()
            fruit = manager.get_fruit(name)
            if fruit is None:
                print(f"{name} 水果不存在")
            else:
                print(f"{name} 水果信息：{fruit.price}元，{fruit.quantity}个")
        
        elif choice == "7":
            print("感谢使用水果超市系统，再见！")
            break
        
        else:
            print("输入错误，请重新输入")

if __name__ == "__main__":
    main()